# Chapter 6: Retrieving Images and Searching Using Image Descriptors

This Jupyter Notebook allows you to interactively edit and run a subset of the code samples from the corresponding chapter in our book, *Learning OpenCV 5 Computer Vision with Python 3*.

Any Jupyter server should be capable of running the Notebook, even if the sample input images files are not available in the server's local filesystem. For example, you can run the Notebook in Google Colab by opening the following link in your Web browser: https://colab.research.google.com/github/PacktPublishing/Learning-OpenCV-5-Computer-Vision-with-Python-Fourth-Edition/blob/main/chapter06/chapter06.ipynb. Specifically, this link opens the Notebook's latest version, hosted on GitHub.

For additional code samples and instructions, please refer to the book and to the GitHub repository at https://github.com/PacktPublishing/Learning-OpenCV-5-Computer-Vision-with-Python-Fourth-Edition.

## Upgrading OpenCV and running the compatibility script

**IMPORTANT:** Run the scripts in this section first and run them in order; otherwise, code in subsequent sections may fail or hang.

If you are running this Notebook in Google Colab or another environment where OpenCV might not be up-to-date, run the following command to upgrade the OpenCV pip package:

In [ ]:
!pip install opencv-contrib-python --upgrade

If the preceding command's output includes a prompt to restart the kernel, do restart it.

Now, run the following script, which provides a compatibility layer between OpenCV and Jupyter:

In [ ]:
# %load ../compat/jupyter_compat.py
import os

import cv2
import numpy
import PIL.Image

from IPython import display
from urllib.request import urlopen


def cv2_imshow(winname, mat):
    mat = mat.clip(0, 255).astype('uint8')
    if mat.ndim == 3:
        if mat.shape[2] == 4:
            mat = cv2.cvtColor(mat, cv2.COLOR_BGRA2RGBA)
        else:
            mat = cv2.cvtColor(mat, cv2.COLOR_BGR2RGB)
    display.display(PIL.Image.fromarray(mat))

cv2.imshow = cv2_imshow


def cv2_waitKey(delay=0):
    return -1

cv2.waitKey = cv2_waitKey


def cv2_imread(filename, flags=cv2.IMREAD_COLOR):
    if os.path.exists(filename):
        image = cv2._imread(filename, flags)
    else:
        url = f'https://github.com/PacktPublishing/Learning-OpenCV-5-Computer-Vision-with-Python-Fourth-Edition/raw/main/*/{filename}'
        resp = urlopen(url)
        image = numpy.asarray(bytearray(resp.read()), dtype='uint8')
        image = cv2.imdecode(image, flags)
    return image

# Cache the original implementation of `imread`, if we have not already
# done so on a previous run of this cell.
if '_imread' not in dir(cv2):
    cv2._imread = cv2.imread

cv2.imread = cv2_imread


What did we just do? We imported OpenCV and we replaced some of OpenCV's I/O functions with our own functions that do not rely on a windowed environment or on a local filesystem.

## Detecting Harris corners

Let's start by finding corners using the Harris corner detection algorithm.

Run the following script, which finds corners in an image of a chessboard:

In [ ]:
# %load corner.py
import cv2

img = cv2.imread('../images/chess_board.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
dst = cv2.cornerHarris(gray, 2, 23, 0.04)
img[dst > 0.01 * dst.max()] = [0, 0, 255]
cv2.imshow('corners', img)
cv2.waitKey()


You probably see that the corners of the squares were detected but many other corners were detected too. Try fine-tuning the parameters of `cornerHarris` to see how the detection results are affected.

## Detecting DoG features and extracting SIFT descriptors

Now, let's experiment with detecting keypoints (specifically, DoG features) and computing keypoint descriptors (specifically, SIFT descriptors).

Run the following script, which performs keypoint detection and description for an image of the beautiful landscape at Varese, Lombady, Italy:

In [ ]:
# %load sift.py
import cv2

img = cv2.imread('../images/varese.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(gray, None)

cv2.drawKeypoints(img, keypoints, img, (51, 163, 236),
                  cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('sift_keypoints', img)
cv2.waitKey()


The visualization shows the scale and orientation of the various SIFT descriptors.

## Using ORB descriptors and brute-force matching

We can apply a brute-force approach to the problem of comparing keypoint descriptors in two images. Thereby, we can find matching keypoints. Let's do so for two images that we describe using the ORB algorithm.

Run the following script, which attempts to find matches between an image of the NASA logo and an image of the Kennedy Space Center:

In [ ]:
# %load orb.py
import cv2
from matplotlib import pyplot as plt

# Load the images.
img0 = cv2.imread('../images/nasa_logo.png',
                  cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread('../images/kennedy_space_center.jpg',
                  cv2.IMREAD_GRAYSCALE)

# Perform ORB feature detection and description.
orb = cv2.ORB_create()
kp0, des0 = orb.detectAndCompute(img0, None)
kp1, des1 = orb.detectAndCompute(img1, None)

# Perform brute-force matching.
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des0, des1)

# Sort the matches by distance.
matches = sorted(matches, key=lambda x:x.distance)

# Draw the best 25 matches.
img_matches = cv2.drawMatches(
    img0, kp0, img1, kp1, matches[:25], img1,
    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# Show the matches.
plt.imshow(img_matches)
plt.show()


Most of those matches are clearly false positives! We will improve on the result in the next section.

## Filtering matches using K-Nearest Neighbors and the ratio test

We can eliminate many false positive matches by performing KNN matching with the ratio test (whereby we reject a dubious "best" match if it is not much better than a "second-best" match).

Run the following script, which first computes a large set of matches and then filters them, again using the NASA logo and Kennedy Space Center as subjects:

In [ ]:
# %load orb_knn.py
import cv2
from matplotlib import pyplot as plt

# Load the images.
img0 = cv2.imread('../images/nasa_logo.png',
                  cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread('../images/kennedy_space_center.jpg',
                  cv2.IMREAD_GRAYSCALE)

# Perform ORB feature detection and description.
orb = cv2.ORB_create()
kp0, des0 = orb.detectAndCompute(img0, None)
kp1, des1 = orb.detectAndCompute(img1, None)

# Perform brute-force KNN matching.
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
pairs_of_matches = bf.knnMatch(des0, des1, k=2)

# Sort the pairs of matches by distance.
pairs_of_matches = sorted(pairs_of_matches, key=lambda x:x[0].distance)

# Draw the 25 best pairs of matches.
img_pairs_of_matches = cv2.drawMatchesKnn(
    img0, kp0, img1, kp1, pairs_of_matches[:25], img1,
    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# Show the pairs of matches.
plt.imshow(img_pairs_of_matches)
plt.show()

# Apply the ratio test.
matches = [x[0] for x in pairs_of_matches
           if len(x) > 1 and x[0].distance < 0.8 * x[1].distance]

# Draw the best 25 matches.
img_matches = cv2.drawMatches(
    img0, kp0, img1, kp1, matches[:25], img1,
    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

# Show the matches.
plt.imshow(img_matches)
plt.show()


Try adjusting the value of the ratio test threshold (from `0.8` to some other value in the range `(0.0, 1.0)`) to see how the results are affected.

## Matching with FLANN

Similarly, we can combine KNN matching and the ratio test with FLANN-based matching instead of brute-force matching.

Run the following script, which finds matches between images of Gaugin paintings:

In [ ]:
# %load flann.py
import numpy as np
import cv2
from matplotlib import pyplot as plt

img0 = cv2.imread('../images/gauguin_entre_les_lys.jpg',
                  cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread('../images/gauguin_paintings.png',
                  cv2.IMREAD_GRAYSCALE)

# Perform SIFT feature detection and description.
sift = cv2.SIFT_create()
kp0, des0 = sift.detectAndCompute(img0, None)
kp1, des1 = sift.detectAndCompute(img1, None)

# Define FLANN-based matching parameters.
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

# Perform FLANN-based matching.
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des0, des1, k=2)

# Prepare an empty mask to draw good matches.
mask_matches = [[0, 0] for i in range(len(matches))]

# Populate the mask based on David G. Lowe's ratio test.
for i, (m, n) in enumerate(matches):
    if m.distance < 0.7 * n.distance:
        mask_matches[i]=[1, 0]

# Draw the matches that passed the ratio test.
img_matches = cv2.drawMatchesKnn(
    img0, kp0, img1, kp1, matches, None,
    matchColor=(0, 255, 0), singlePointColor=(255, 0, 0),
    matchesMask=mask_matches, flags=0)

# Show the matches.
plt.imshow(img_matches)
plt.show()


Try adjusting the FLANN parameters in `index_params` and `search_params` to see how the results are affected.

## Finding homography with FLANN-based matches

Let's explore the perspective relationship between two matching images (or matching parts of images) by finding the homography between the matching keypoints.

Run the following script, which finds the homography between a small image of a tattoo and a larger image of a hand bearing the same tattoo:

In [ ]:
# %load homography.py
import numpy as np
import cv2
from matplotlib import pyplot as plt

MIN_NUM_GOOD_MATCHES = 10

img0 = cv2.imread('../images/tattoos/query.png',
                  cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread('../images/tattoos/anchor-man.png',
                  cv2.IMREAD_GRAYSCALE)

# Perform SIFT feature detection and description.
sift = cv2.SIFT_create()
kp0, des0 = sift.detectAndCompute(img0, None)
kp1, des1 = sift.detectAndCompute(img1, None)

# Define FLANN-based matching parameters.
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

# Perform FLANN-based matching.
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des0, des1, k=2)

# Find all the good matches as per Lowe's ratio test.
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

if len(good_matches) >= MIN_NUM_GOOD_MATCHES:
    src_pts = np.float32(
        [kp0[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32(
        [kp1[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    mask_matches = mask.ravel().tolist()

    h, w = img0.shape
    src_corners = np.float32(
        [[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    dst_corners = cv2.perspectiveTransform(src_corners, M)
    dst_corners = dst_corners.astype(np.int32)

    # Draw the bounds of the matched region based on the homography.
    num_corners = len(dst_corners)
    for i in range(num_corners):
        x0, y0 = dst_corners[i][0]
        if i == num_corners - 1:
            next_i = 0
        else:
            next_i = i + 1
        x1, y1 = dst_corners[next_i][0]
        cv2.line(img1, (x0, y0), (x1, y1), 255, 3, cv2.LINE_AA)

    # Draw the matches that passed the ratio test.
    img_matches = cv2.drawMatches(
        img0, kp0, img1, kp1, good_matches, None,
        matchColor=(0, 255, 0), singlePointColor=None,
        matchesMask=mask_matches, flags=2)

    # Show the homography and good matches.
    plt.imshow(img_matches)
    plt.show()
else:
    print("Not enough matches good were found - %d/%d" % \
          (len(good_matches), MIN_MATCH_COUNT))


Try adjusting the parameters of `findHomography` to see how the results are affected.

## Summary

That is all for now! Please refer to the book and to the GitHub repository for additional samples involving keypoint descriptors (including the SURF algorithm) and searching for matching images.